# Calculate velocity and curvature

Within this notebook it shall be shown how one can calculate velocity and curvatures values from a particle set created by DaVis. In order to do so, the Python library **lvpyio** is used to import the trajectory data.

In [1]:
# Import necessary packages
import numpy as np
import lvpyio as lv
from csaps import CubicSmoothingSpline

## Define function for velocity and curvature calculation

In [2]:
# This is a function, which fits a single trajectory with a smoothing cubic spline. 
# Based on this spline, the first and second derivatives of the trajectories can be calculated.
# The first derivated is used to calculate the particle's velocity.
# Additionally the local curvature of the trajectory is calculated using both the first and second derivative.

def f_calc_scalars(track,dt,scaling, smooth_val):

    # get coordinates and rescale to meters
    x_raw = (track.particles[:]['x']*scaling.x.slope+scaling.x.offset)/1000
    y_raw = (track.particles[:]['y']*scaling.y.slope+scaling.y.offset)/1000
    z_raw = (track.particles[:]['z']*scaling.z.slope+scaling.z.offset)/1000
    
    # get track length
    length_track = np.size(x_raw)
    
    # Create arbitrary time vector. Absolute values not necessary here, only the spacing with dt
    t_raw=np.linspace(-length_track/2,length_track/2,length_track)*dt # create time vector for filter window
    
    # Fit x-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, x_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dx1 = s.derivative(nu=1)
    dx2 = s.derivative(nu=2)
    # Save velocity
    U = dx1(t_raw)
    
    # Fit y-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, y_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dy1 = s.derivative(nu=1)
    dy2 = s.derivative(nu=2)
    # Save velocity
    V = dy1(t_raw)
    
    # Fit z-data with Smoothing spline
    s = CubicSmoothingSpline(t_raw, z_raw, smooth=smooth_val, normalizedsmooth=True).spline
    # Calculate derivatives
    dz1 = s.derivative(nu=1)
    dz2 = s.derivative(nu=2)
    # Save velocity
    W = dz1(t_raw)
    
    # Build vectors of derivatives    
    d1vec = np.c_[dx1(t_raw),dy1(t_raw)]
    d1vec = np.c_[d1vec, dz1(t_raw)]

    d2vec = np.c_[dx2(t_raw),dy2(t_raw)]
    d2vec = np.c_[d2vec, dz2(t_raw)]
    
    # Calculate curvature (see equation below curvature = kappa)
    curvature = np.linalg.norm(np.cross(d1vec[None, :],d2vec), axis=2) / np.linalg.norm(d1vec,axis=1)**3
    
    return curvature, U, V, W

For the definition of curvature $\kappa$ see https://en.wikipedia.org/wiki/Curvature and mentioned references:
<br>
<br>
<div align="center"><b> $\kappa = \dfrac{||\dot{x} \times \ddot{x}||}{ ||\dot{x}||^3}$ </b></div>


## Add new scalars to the set

Now, the newly calculated properties can be attached to the particle set.

In [3]:
# Read DaVis particle set
tracks_lv = lv.read_particles('data\\Tracks_Cylinder_Wake')

dt = tracks_lv.times()[1]-tracks_lv.times()[0] # retrieve dt of measurements

# initialize new list, which will contain the tracks with the added scalars
tracks_export = list()

# read scaling of positional data
scaling = tracks_lv.scales

# define scales of new scalars
scalar_scales = {"Curvature": lv.Scale(1, 0, "1/m", "local curvature"),"U": lv.Scale(1, 0, "m/s", "Spline velocity u"),"V": lv.Scale(1, 0, "m/s", "Spline velocity v"),"W": lv.Scale(1, 0, "m/s", "Spline velocity w")}

for ii in range(tracks_lv.track_count): # loop over all trajectories

    track_temp = tracks_lv.single_track(ii) # get trajectory information
    curvature, U, V, W = f_calc_scalars(track_temp,dt,scaling, 0.85) # calculate new scalars
    track_temp.scalars = {"Curvature": curvature[0,:],"U": U,"V": V,"W": W} # add scalars to track object
    
    tracks_export.append(track_temp) # add track to export list
    

C:\Users\tjanke\AppData\Local\Temp\ipykernel_14420\663148827.py:51: RuntimeWarning: invalid value encountered in divide
  curvature = np.linalg.norm(np.cross(d1vec[None, :],d2vec), axis=2) / np.linalg.norm(d1vec,axis=1)**3


## Write set

In [4]:
# Write new particle set with same properties as the original one.
# But now, the new scalars have been added to each trajectory
lv.write_particles(dict(times=tracks_lv.times(), tracks=tracks_export, 
                        scales = tracks_lv.scales, scalar_scales = scalar_scales, 
                        attributes = tracks_lv.attributes, bounds = tracks_lv.bounds), "data\\tracks_with_scalar")